In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.plotly as py
import plotly.graph_objs as go
import plotly
from plotly.offline import init_notebook_mode
import plotly.graph_objs as go
plotly.offline.init_notebook_mode(connected=True)

In [2]:
training_df = pd.read_csv('~/train.csv', error_bad_lines=False)

b'Skipping line 27302: expected 6 fields, saw 10\nSkipping line 47605: expected 6 fields, saw 9\nSkipping line 88183: expected 6 fields, saw 8\nSkipping line 108360: expected 6 fields, saw 8\nSkipping line 115147: expected 6 fields, saw 7\n'
b'Skipping line 155454: expected 6 fields, saw 9\nSkipping line 182251: expected 6 fields, saw 7\nSkipping line 202290: expected 6 fields, saw 7\nSkipping line 215639: expected 6 fields, saw 10\nSkipping line 235670: expected 6 fields, saw 8\n'
b'Skipping line 262254: expected 6 fields, saw 7\nSkipping line 295488: expected 6 fields, saw 9\nSkipping line 322124: expected 6 fields, saw 7\nSkipping line 382155: expected 6 fields, saw 7\n'
b'Skipping line 408864: expected 6 fields, saw 9\nSkipping line 435529: expected 6 fields, saw 7\nSkipping line 442176: expected 6 fields, saw 7\nSkipping line 468715: expected 6 fields, saw 9\nSkipping line 482042: expected 6 fields, saw 7\nSkipping line 495357: expected 6 fields, saw 7\nSkipping line 502004: expec

In [3]:
training_df.shape

(774497, 6)

In [4]:
test_df = pd.read_csv('~/test.csv')
print("Number of testing data: %d" %len(test_df))
test_df.head(5)

/home/ubuntu/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning:

Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.



Number of testing data: 3563475


,test_id,question1,question2
0,0,How does the Surface Pro himself 4 compare wit...,Why did Microsoft choose core m3 and not core ...
1,1,Should I have a hair transplant at age 24? How...,How much cost does hair transplant require?
2,2,What but is the best way to send money from Ch...,What you send money to China?
3,3,Which food not emulsifiers?,What foods fibre?
4,4,"How ""aberystwyth"" start reading?",How their can I start reading?


In [5]:
test_df.shape

(3563475, 3)

In [231]:
y =training_df['is_duplicate']

In [7]:
question_corpus = pd.concat([training_df[['question1','question2']], test_df[['question1','question2']]])

In [8]:
from nltk.corpus import stopwords
from nltk.stem.porter import *
import nltk
nltk.download("stopwords")
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /home/ubuntu/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /home/ubuntu/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [21]:
from nltk.stem import PorterStemmer

In [9]:

sw_dict={}
    
for i in stopwords.words('english'):
    sw_dict[i]=1

In [22]:
import re
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer


lemi_words=[]
stem_words=[]
lmtzr = nltk.WordNetLemmatizer().lemmatize
stemmer = PorterStemmer()



def do_lemitise(word):
    old_word=word
    word = lmtzr(word)
    if word != old_word:
        lemi_words.append((old_word, word))
        
    return word
    
def do_stem(word):
    old_word=word
    word = stemmer.stem(word)
    if word != old_word:
        stem_words.append((old_word, word))
    return word
        

def do_process_word(word):
    word = do_stem(word)
    word = do_lemitise(word)
    return word
    
    
    

def corpus_filteration(dfList):
    
    actual_list =[]
    

    
    item_num=0

    for k, v in dfList.iteritems():
        questions_corpus = re.sub('[^a-zA-Z]',' ', str(v)).lower().split()
        questions_corpus = [do_process_word(word) for word in questions_corpus if word not in sw_dict.keys()]
        
        actual_list.append(questions_corpus)
        item_num = item_num + 1
        if (item_num%1000000 == 0):
            print ("Done so far {} ".format(item_num) )
    
    return actual_list

In [23]:
training_df_question1 = corpus_filteration(training_df['question1'])

In [51]:
training_df_question1[:5]

[['step', 'step', 'guid', 'invest', 'share', 'market', 'india'],
 ['stori', 'kohinoor', 'koh', 'noor', 'diamond'],
 ['increas', 'speed', 'internet', 'connect', 'use', 'vpn'],
 ['mental', 'lone', 'solv'],
 ['one',
  'dissolv',
  'water',
  'quikli',
  'sugar',
  'salt',
  'methan',
  'carbon',
  'di',
  'oxid']]

In [24]:
training_df_question2 = corpus_filteration(training_df['question2'])

In [25]:
question_corpus_list= corpus_filteration(question_corpus['question1']) + corpus_filteration(question_corpus['question2'])

Done so far 1000000 
Done so far 2000000 
Done so far 3000000 
Done so far 4000000 
Done so far 1000000 
Done so far 2000000 
Done so far 3000000 
Done so far 4000000 


In [28]:
from gensim.models import Word2Vec
model = Word2Vec(question_corpus_list, min_count=1,size=50)



words = list(model.wv.vocab)

model.save('model.bin')

In [29]:
from gensim.models import Word2Vec
from gensim.models import KeyedVectors

word2vec_model = KeyedVectors.load('model.bin')
word2vec_model.init_sims(replace=True)

In [31]:
from pyemd import emd
def distance_calculation(d1,d2):
        i=j=0
        distance_list=[]
        len_train_01 =len(d1)
        len_train_02 =len(d2)
        item_num=0
        
        
        while (i<len_train_01 and j<len_train_02):
            distance_questions = word2vec_model.wv.wmdistance(d1[i], d2[j])
            
            distance_list.append(distance_questions)
            i=i+1
            j=j+1
            item_num = item_num + 1
            if (item_num%100000 == 0):
                print ("Done so far {} ".format(item_num) )
        training_df['is_duplicate_01']=distance_list
        return training_df['is_duplicate_01']

In [ ]:
distance_calculation(training_df_question1,training_df_question2)

In [32]:
import pandas as pd
new_df = pd.DataFrame(distance_calculation(training_df_question1,training_df_question2), columns=['cos_dis'])

Done so far 100000 
Done so far 200000 
Done so far 300000 
Done so far 400000 
Done so far 500000 
Done so far 600000 
Done so far 700000 


In [34]:
len(new_df)

0

In [35]:

p=distance_calculation(training_df_question1,training_df_question2)

Done so far 100000 
Done so far 200000 
Done so far 300000 
Done so far 400000 
Done so far 500000 
Done so far 600000 
Done so far 700000 


In [128]:
p

0         0.181683
1         0.810330
2         0.548800
3         1.226094
4         0.916621
5         0.592209
6         1.351881
7         0.444190
8         0.000000
9         0.627300
10        1.199619
11        0.570249
12        0.000000
13        0.307694
14        0.084350
15        0.768795
16        0.000000
17        0.412530
18        0.564870
19        0.125863
20        0.654571
21        0.309979
22        0.000000
23        1.281433
24        0.772966
25        0.196684
26        0.331694
27        0.788021
28        0.245634
29        0.655290
            ...   
774467    0.587871
774468    0.307430
774469    0.294338
774470    0.563848
774471    0.825117
774472    0.385414
774473    0.939958
774474    0.431749
774475    0.920646
774476    0.490241
774477    0.000000
774478    0.805013
774479    0.706638
774480    0.512749
774481    0.496723
774482    0.672312
774483    0.172159
774484    0.989171
774485    0.506622
774486    0.818382
774487    0.198712
774488    0.

In [44]:
cols=['cos_dis']
new_df123 = pd.DataFrame([p])

In [129]:
train_data11=p.to_frame(name='distance_calculation')

In [131]:
train_data11

,distance_calculation
0,0.181683
1,0.810330
2,0.548800
3,1.226094
4,0.916621
5,0.592209
6,1.351881
7,0.444190
8,0.000000
9,0.627300


In [71]:
def len_question(list1,list2):
    len_corpus=774497
    i=0
    j=0
    result=[]
    print(list1[i])
    print(list2[j])
    
    for i in range(0,len(list1)):
        
        if len(list1[i]) > len(list2[j]):
            result.append(len(list1[i])-len(list2[j]))
        elif len(list2[j]) > len(list1[i]):
            result.append(len(list2[j])-len(list1[i]))
        else:
            result.append(0)
        j=j+1
    return result
    

In [69]:
len(training_df_question1)

774497

In [73]:
len_features=len_question(training_df_question1,training_df_question2)

['step', 'step', 'guid', 'invest', 'share', 'market', 'india']
['step', 'step', 'guid', 'invest', 'share', 'market']


In [132]:
train_data12 = pd.DataFrame(len_features, columns=['len_features'])

In [133]:
train_data12

,len_features
0,1
1,5
2,1
3,2
4,5
5,1
6,5
7,0
8,0
9,1


In [139]:
def common_word(list1,list2):
    result=[]
    ct=0
    for i in range(0,len(list1)):
        if len(list1[i])>len(list2[i]):
            s1=set(list2[i])
            for j in list1[i]:
                if j in s1:
                    ct=ct+1
                else:
                    continue
            tg=ct/(len(list1[i])+len(list2[i]))
            result.append(tg)
            ct=0
            tg=0
            s1=set()
        elif len(list2[i])>len(list1[i]):
            s1=set(list1[i])
            for j in list2[i]:
                if j in s1:
                    ct=ct+1
                else:
                    continue
            tg=ct/(len(list1[i])+len(list2[i]))
            result.append(tg)
            ct=0
            tg=0
            s1=set()
        else:
            result.append(0)
    return result

In [140]:
common_w1 =common_word(training_df_question1,training_df_question2)

In [141]:
common_w1

[0.46153846153846156,
 0.26666666666666666,
 0.2727272727272727,
 0.0,
 0.13333333333333333,
 0.26666666666666666,
 0.0,
 0,
 0,
 0.2727272727272727,
 0,
 0.2857142857142857,
 0,
 0.42857142857142855,
 0,
 0.15789473684210525,
 0,
 0.3333333333333333,
 0.26666666666666666,
 0,
 0.2857142857142857,
 0,
 0,
 0.0,
 0,
 0.46153846153846156,
 0.4,
 0.25,
 0,
 0.2727272727272727,
 0,
 0.3333333333333333,
 0.46153846153846156,
 0.0,
 0.42857142857142855,
 0.3333333333333333,
 0.3333333333333333,
 0.13636363636363635,
 0.2,
 0,
 0.16666666666666666,
 0,
 0,
 0.2222222222222222,
 0.375,
 0.3333333333333333,
 0.0,
 0.0,
 0.3333333333333333,
 0,
 0,
 0.5384615384615384,
 0.0,
 0.2222222222222222,
 0.0,
 0.1111111111111111,
 0.1111111111111111,
 0.2857142857142857,
 0.3157894736842105,
 0.2857142857142857,
 0.2222222222222222,
 0,
 0.14285714285714285,
 0.42857142857142855,
 0,
 0.36363636363636365,
 0,
 0.3333333333333333,
 0.4,
 0.0,
 0.3333333333333333,
 0,
 0.42857142857142855,
 0.222222222222

In [134]:
type(common_w)

list

In [144]:
train_data14 = pd.DataFrame(common_w1, columns=['common_w1'])

In [145]:
train_data14

,common_w1
0,0.461538
1,0.266667
2,0.272727
3,0.000000
4,0.133333
5,0.266667
6,0.000000
7,0.000000
8,0.000000
9,0.272727


In [92]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [93]:
vectorizer  = TfidfVectorizer()

In [101]:
def tfidf(list1,list2):
    response =0
    result=[]
    for i in range(0,len(list1)):
        response =vectorizer.fit_transform(list1[i],list2[i])
        result.append(response)
        
        
    return result 

In [102]:
tfidf_col = tfidf(training_df_question1[:2],training_df_question2[:2])

In [103]:
tfidf_col

[<7x6 sparse matrix of type '<class 'numpy.float64'>'
 	with 7 stored elements in Compressed Sparse Row format>,
 <5x5 sparse matrix of type '<class 'numpy.float64'>'
 	with 5 stored elements in Compressed Sparse Row format>]

In [106]:
from collections import Counter

In [107]:
train_questions = pd.Series(training_df['question1'].tolist() + training_df['question2'].tolist()).astype(str)
test_questions = pd.Series(test_df['question1'].tolist() + test_df['question2'].tolist()).astype(str)
#Implement TFIDF function
def get_weight(count, eps=5000, min_count=2):
    if count < min_count:
        return 0 #remove words only appearing once 
    else:
        R = 1.0 / (count + eps)
        return R

eps = 5000 
words = (" ".join(train_questions)).lower().split()
counts = Counter(words)
weights = {word: get_weight(count) for word, count in counts.items()}
#print("Most common words: ", (sorted(weights.items(), key=lambda x: x[1] if x[1] > 0 else 9999)[:10]))

In [108]:
def tfidf_word_share_norm(x):
    w1 = set(map(lambda word: word.lower().strip(), str(x['question1']).split(" ")))
    w2 = set(map(lambda word: word.lower().strip(), str(x['question2']).split(" "))) 
    if len(w1)==0 or len(w2)==0:
        return 0
    common = w1 & w2
    share_weight = [weights.get(word, 0) for word in common]
    total_weight = [weights.get(word, 0) for word in w1]+[weights.get(word, 0) for word in w2]
    return np.sum(share_weight)/np.sum(total_weight)

In [148]:
train_data14['TFIDF_share'] = pd.DataFrame((training_df.apply(tfidf_word_share_norm, axis=1, raw=True)),columns=['tfidf'])

In [149]:
train_data14.head()

,common_w1,TFIDF_share
0,0.461538,0.404563
1,0.266667,0.208701
2,0.272727,0.141290
3,0.000000,0.000000
4,0.133333,0.005792


In [150]:
train_data12.head()

,len_features
0,1
1,5
2,1
3,2
4,5


In [151]:
train_data14['len_features']=train_data12['len_features']

In [152]:
train_data11.head()

,distance_calculation
0,0.181683
1,0.810330
2,0.548800
3,1.226094
4,0.916621


In [153]:
train_data14['distance_calculation']=train_data11['distance_calculation']

In [193]:
train_data14.head()

,index,common_w1,TFIDF_share,len_features,distance_calculation
0,0,0.461538,0.404563,1,0.181683
1,1,0.266667,0.208701,5,0.810330
2,2,0.272727,0.141290,1,0.548800
3,3,0.000000,0.000000,2,1.226094
4,4,0.133333,0.005792,5,0.916621


In [155]:
len(train_data14)

774497

In [199]:
train_data14 = train_data14.reset_index(drop=True)

In [227]:
train_data14.head()

,index,common_w1,TFIDF_share,len_features,distance_calculation
0,0,0.461538,0.404563,1,0.181683
1,1,0.266667,0.208701,5,0.810330
2,2,0.272727,0.141290,1,0.548800
3,3,0.000000,0.000000,2,1.226094
4,4,0.133333,0.005792,5,0.916621


In [228]:
import numpy as np
np.sum(np.isnan(train_data14))


index                   0
common_w1               0
TFIDF_share             0
len_features            0
distance_calculation    0
dtype: int64

In [232]:
np.sum(np.isnan(y))


35

In [239]:
y=y.to_frame()

In [240]:
type(y)

pandas.core.frame.DataFrame

In [241]:
y.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 774497 entries, 0 to 774496
Data columns (total 1 columns):
is_duplicate    774462 non-null float64
dtypes: float64(1)
memory usage: 5.9 MB


In [242]:
y.is_duplicate.unique()

array([ 0.,  1., nan])

In [243]:
y=y.fillna(0)

In [244]:
y.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 774497 entries, 0 to 774496
Data columns (total 1 columns):
is_duplicate    774497 non-null float64
dtypes: float64(1)
memory usage: 5.9 MB


In [233]:
train_data14.head()

,index,common_w1,TFIDF_share,len_features,distance_calculation
0,0,0.461538,0.404563,1,0.181683
1,1,0.266667,0.208701,5,0.810330
2,2,0.272727,0.141290,1,0.548800
3,3,0.000000,0.000000,2,1.226094
4,4,0.133333,0.005792,5,0.916621


In [245]:
X = train_data14[['common_w1','TFIDF_share','len_features','distance_calculation']]

In [252]:
np.isnan(y)

,is_duplicate
0,False
1,False
2,False
3,False
4,False
5,False
6,False
7,False
8,False
9,False


In [253]:
X[X==np.inf]=np.nan
X.fillna(X.mean(), inplace=True)

/home/ubuntu/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy

/home/ubuntu/anaconda3/lib/python3.6/site-packages/pandas/core/frame.py:2554: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy

/home/ubuntu/anaconda3/lib/python3.6/site-packages/pandas/core/generic.py:4355: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy



In [254]:
from sklearn.model_selection import train_test_split

X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=42)

In [255]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV

knn = KNeighborsClassifier()
grid = {
    'n_neighbors': list(range(2, 10, 2)),
    'weights': ['uniform', 'distance']
}
cv = GridSearchCV(knn, grid, scoring='neg_log_loss', verbose = True)
cv.fit(X_train, y_train)
#print(cv.best_params_)
score_knn = cv.best_estimator_.score(X_valid, y_valid)
print("K Nearest Neighbor model accuracy: %.3f" %score_knn)

Fitting 3 folds for each of 8 candidates, totalling 24 fits


/home/ubuntu/anaconda3/lib/python3.6/site-packages/sklearn/model_selection/_validation.py:458: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().

/home/ubuntu/anaconda3/lib/python3.6/site-packages/sklearn/model_selection/_validation.py:458: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().

/home/ubuntu/anaconda3/lib/python3.6/site-packages/sklearn/model_selection/_validation.py:458: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().

/home/ubuntu/anaconda3/lib/python3.6/site-packages/sklearn/model_selection/_validation.py:458: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example usin

K Nearest Neighbor model accuracy: 0.931


In [257]:
y_pred = cv.predict(X_valid)

In [259]:
from sklearn.metrics import accuracy_score
print(accuracy_score(y_valid,y_pred))

0.9309360877985797


#random forest

In [262]:
from sklearn.ensemble import RandomForestClassifier
#Benchmark model : random forest
rfc = RandomForestClassifier(max_depth=5, random_state=0)
rfc.fit(X_train,y_train)
prediction = rfc.predict_proba(X_valid)
pre = rfc.predict(X_valid)
score_rfc = rfc.score(X_valid, y_valid)
print("Random Forest (Benchmark model) has accuracy: %.3f" %score_rfc)

/home/ubuntu/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:4: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().



Random Forest (Benchmark model) has accuracy: 0.718


#xgboost

In [268]:
#Model 6: XGBoost 
import xgboost as xgb

params = {}
params['objective'] = 'binary:logistic'
params['eval_metric'] = 'logloss'
params['eta'] = 0.02
params['max_depth'] = 4

d_train = xgb.DMatrix(X_train, label=y_train)
d_valid = xgb.DMatrix(X_valid, label=y_valid)

watchlist = [(d_train, 'train'), (d_valid, 'valid')]
bst = xgb.train(params, d_train, 500, watchlist, early_stopping_rounds=100, verbose_eval=10)


[0]	train-logloss:0.686841	valid-logloss:0.687011
Multiple eval metrics have been passed: 'valid-logloss' will be used for early stopping.

Will train until valid-logloss hasn't improved in 100 rounds.
[10]	train-logloss:0.636604	valid-logloss:0.636592
[20]	train-logloss:0.601017	valid-logloss:0.600828
[30]	train-logloss:0.574628	valid-logloss:0.574515
[40]	train-logloss:0.554862	valid-logloss:0.55457
[50]	train-logloss:0.53968	valid-logloss:0.539363
[60]	train-logloss:0.527891	valid-logloss:0.527571
[70]	train-logloss:0.518673	valid-logloss:0.518354
[80]	train-logloss:0.511448	valid-logloss:0.51112
[90]	train-logloss:0.505663	valid-logloss:0.505401
[100]	train-logloss:0.501098	valid-logloss:0.500818
[110]	train-logloss:0.49746	valid-logloss:0.497146
[120]	train-logloss:0.494429	valid-logloss:0.494188
[130]	train-logloss:0.492056	valid-logloss:0.491823
[140]	train-logloss:0.490167	valid-logloss:0.489898
[150]	train-logloss:0.488548	valid-logloss:0.488329
[160]	train-logloss:0.48716	val